In [3]:
import os 
from glob import glob 
from os import path

from itertools import groupby, filterfalse

import pvl
import zlib

import importlib
import os
from glob import glob

from abc import ABC, abstractmethod

os.environ['SPICE_DATA'] = '/data/spice'

def get_metakernels(missions=set(), years=set(), versions=set()):
    """
    Mostly doing filtering here, might be worth using Pandas here?
    
    Super beta, proof of concept code slapped together at a Starbucks.
    """
    spice_dir = os.environ.get("SPICE_DATA")
    if spice_dir is None:
        raise Exception("$SPICE_DATA not set")
    
    if isinstance(missions, str):
        missions = {missions}
        
    if isinstance(years, str) or isinstance(years, int):
        years = {str(years)}
    else:
        years = {str(year) for year in years}
    
    avail = {
        'count': 0,
        'data': []
    }
    
    mission_dirs = list(filter(os.path.isdir, glob(os.path.join(spice_dir, '*'))))
    
    for md in mission_dirs:
        # Assuming spice root has the same name as the original on NAIF website"
        mission = os.path.basename(md).split('-')[0]
        if missions and mission not in missions:
            continue
        
        metakernel_keys = ['mission', 'year', 'version', 'path']
        
        # recursive glob to make metakernel search more robust to 
        # subtle directory structure differences 
        metakernel_paths = sorted(glob(os.path.join(md, '**','*.tm'), recursive=True))
        
        metakernels = [dict(zip(metakernel_keys, [mission]+path.splitext(path.basename(k))[0].split('_')[1:3] + [k])) for k in metakernel_paths]
        
        # naive filter, do we really need anything else?
        if years:
            metakernels = list(filter(lambda x:x['year'] in years, metakernels))
        if versions:
            if versions == 'latest':  
                latest = []
                # Panda's groupby is overrated 
                for k, g in groupby(metakernels, lambda x:x['year']):
                    items = list(g)
                    latest.append(max(items, key=lambda x:x['version']))
                metakernels = latest
            else:        
                metakernels = list(filter(lambda x:x['version'] in versions, metakernels))
            
        avail['data'].extend(metakernels)
    
    avail['count'] = len(avail['data'])
    if not avail:
        avail = {
            'count' : 0,
            'data' : 'ERROR: NONE OF {} ARE VALID MISSIONS'.format(missions)
        }
        
    return avail


    

class Base(ABC):
    """
    Probably worth splitting these into mixins if we can 
    """
    def __init__(self, label, *args, **kwargs):
        self.label = label
    
    @property 
    def lines(self, label=None):
        return label['LINES']

    @property 
    def samples(self, label=None):
        return label['SAMPLES']
    
    @property
    def et(self):
        sclock = self.label['SPACECRAFT_CLOCK_START_COUNT']
        exposure_duration = self.label['EXPOSURE_DURATION'].value
        exposure_duration = exposure_duration * 0.001  # Scale to seconds

        # Get the instrument id, and, since this is a framer, set the time to the middle of the exposure
        start_et = spice.scs2e(self.spacecraft_id, sclock)
        start_et += (exposure_duration / 2.0)
        end_et = spice.scs2e(self.spacecraft_id, self.label['SPACECRAFT_CLOCK_STOP_COUNT']) + (exposure_duration / 2.0)
        et = (start_et + end_et)/2
        return et
    
    @property
    def del_et(self):
        sclock = self.label['SPACECRAFT_CLOCK_START_COUNT']
        exposure_duration = self.label['EXPOSURE_DURATION'].value
        exposure_duration = exposure_duration * 0.001  # Scale to seconds

        # Get the instrument id, and, since this is a framer, set the time to the middle of the exposure
        start_et = spice.scs2e(spacecraft_id, sclock)
        start_et += (exposure_duration / 2.0)
        end_et = spice.scs2e(spacecraft_id, self.label['SPACECRAFT_CLOCK_STOP_COUNT']) + (exposure_duration / 2.0)
        return end_et - start_et

    @property
    def start_et(self):
        sclock = self.label['SPACECRAFT_CLOCK_START_COUNT']
        exposure_duration = self.label['EXPOSURE_DURATION'].value
        exposure_duration = exposure_duration * 0.001  # Scale to seconds

        # Get the instrument id, and, since this is a framer, set the time to the middle of the exposure
        return spice.scs2e(spacecraft_id, sclock)
 
    @property
    def spacecraft_name(self):
        return self.label['MISSION_NAME']
    
    @property
    def instrument_id(self):
        id_lookup = {
            'MDIS-NAC':'MSGR_MDIS_NAC',
            'MERCURY DUAL IMAGING SYSTEM NARROW ANGLE CAMERA':'MSGR_MDIS_NAC',
            'MERCURY DUAL IMAGING SYSTEM WIDE ANGLE CAMERA':'MSGR_MDIS_WAC'
        }
        return self.label['INSTRUMENT_ID']
    
    @property
    def ikid(self):
        return spice.bods2c(self.instrument_id)

    @property
    def spacecraft_id(self):
        return spice.bods2c(self.spacecraft_name)

    @property
    def odkx(self):
        return spice.gdpool('INS{}_OD_T_X'.format(self.ikid),0, 10)
    
    @property 
    def odky(self):
        return spice.gdpool('INS{}_OD_T_Y'.format(self.ikid), 0, 10)
    
    @property 
    def focal2pixel_lines(self):
        return list(spice.gdpool('INS{}_TRANSX'.format(self.ikid), 0, 3))
    
    @property 
    def focal2pixels_samples(self):
        list(spice.gdpool('INS{}_TRANSX'.format(self.ikid), 0, 3))
        
    @property 
    def focal_length(self):
        return float(spice.gdpool('INS{}_FOCAL_LENGTH'.format(self.ikid), 0, 1)[0])
            
    @property 
    def focal_length_epsilon(self):
        return float(spice.gdpool('INS{}_FL_UNCERTAINTY'.format(self.ikid), 0, 1)[0])

    @property 
    def semimajor(self):
        rad = spice.bodvrd(self.label['TARGET_NAME'], 'RADII', 3)
        return rad[1][1]

    @property
    def semiminor(self):
        rad = spice.bodvrd(self.label['TARGET_NAME'], 'RADII', 3)
        return rad[1][0]

    @property
    def reference_frame(self):
        return 'IAU_{}'.format(self.label['TARGET_NAME'])
    
    @property
    def sun_position(self):
        sun_state, _ = spice.spkezr("SUN",
                                     self.et,
                                     self.reference_frame,
                                     'NONE',
                                     self.label['TARGET_NAME'])

        return sun_state[:4]

    @property
    def sun_velocity(self):
        sun_state, lt = spice.spkezr("SUN",
                                     self.et,
                                     self.reference_frame,
                                     'NONE',
                                     self.label['TARGET_NAME'])

        return sun_state[4:7]
    
    @property
    def sensor_velocity(self):
        v_state, _ = spice.spkezr(mission_name,
                                           self.et,
                                           self.reference_frame,
                                           'None',
                                           self.label['TARGET_NAME'])
        return vstate[3:6]
    @property
    def sensor_position(self):
        loc, _ = spice.spkpos(self.label['TARGET_NAME'], 
                              self.et, 
                              self.reference_frame, 
                              'None', 
                              self.spacecraft_name)
        return loc[:4]
    


In [4]:
from glob import glob
import os

import pvl
import spiceypy as spice
import numpy as np

class Messenger(Base):
    @property
    def instrument_id(self):
        id_lookup = {
            'MDIS-NAC':'MSGR_MDIS_NAC',
            'MERCURY DUAL IMAGING SYSTEM NARROW ANGLE CAMERA':'MSGR_MDIS_NAC',
            'MERCURY DUAL IMAGING SYSTEM WIDE ANGLE CAMERA':'MSGR_MDIS_WAC'
        }
        return id_lookup[self.label['INSTRUMENT_ID']]
        
    @property
    def focal_length(self):
        """
        """
        coeffs = spice.gdpool('INS{}_FL_TEMP_COEFFS '.format(self.ikid), 0, 5)

        # reverse coeffs, mdis coeffs are listed a_0, a_1, a_2 ... a_n where
        # numpy wants them a_n, a_n-1, a_n-2 ... a_0
        f_t = np.poly1d(coeffs[::-1])

        # eval at the focal_plane_tempature
        return f_t(self.label['FOCAL_PLANE_TEMPERATURE'].value)

class Cassini(Base):
    """
    So Cassini doesn't have a distortion model, so calling to get odkx/odky should just fail? 
    Or do explicitly have to write a function whihc return None? 
    """

    @property
    def instrument_id(self):
        instrument_names = {
            "ISSNA" : "CASSINI_ISS_NAC",
            "ISSWA" : "CASSINI_ISS_WAC"
        }
        return self.label['INSTRUMENT_ID']

In [8]:
# lol 
latest_mess = get_metakernels(missions='mess', versions='latest', years=2013)

# Should Drivers be responsible for loading/unloading kernels? 
spice.furnsh(latest_mess['data'][0]['path'])

d = Messenger(label=pvl.load('/data/spice/EN1007907102M.IMG.lbl'))
d.focal_length, d.spacecraft_id, d.odkx, d.sun_position, d.semimajor, d.sensor_position

(549.23479652106016,
 -236,
 array([  0.00000000e+00,   1.00185427e+00,   0.00000000e+00,
          0.00000000e+00,  -5.09444047e-04,   0.00000000e+00,
          1.00401047e-05,   0.00000000e+00,   1.00401047e-05,
          0.00000000e+00]),
 array([ -3.16407024e+07,  -6.06380937e+07,  -3.87311046e+04,
         -3.81911239e+01]),
 2439.4000000000001,
 array([-1728.37368696,  2088.42341073, -2082.86052983]))